# Práctico 1 - Parte 2 de 3

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

**Implementación de red neuronal [Perceptrón Multicapa](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP).**

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

En esta segunda parte se preprocesan los datos y se los guarda para ser utilizados en la tercera parte, en la que se arma el dataset y se entrena y prueba el modelo.

## ⚠ IMPORTANTE ⚠

Por favor leer el archivo [Practico_1.md](https://github.com/grobiglio/deepleaning/blob/master/practico/Practico_1.md#deep-learning---trabajo-pr%C3%A1ctico-1) que se encuentra en el repositorio donde se puso este trabajo práctico.

## Importaciones

In [1]:
import bz2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from gensim import corpora
import pandas as pd
from tqdm.notebook import tqdm
from practico1_modulo import *

## Constantes

In [2]:
ARCHIVO_SET_DE_ENTRENAMIENTO = './data/training_set.csv'
ARCHIVO_SET_DE_PRUEBA = './data/test_set.csv'
ARCHIVO_SET_DE_VALIDACION = './data/validation_set.csv'
ARCHIVO_DE_EMBEDDINGS = './data/SBW-vectors-300-min5.txt.bz2'
ARCHIVO_DICCIONARIO = './data/diccionario.txt'
EPOCHS = 5
VOCAB_SIZE = 50000 # Cantidad de palabras en el diccionario
MUESTRAS = 0 # Cantidad de muestras que se toman del set de entrenamiento
             # Si se configura valor 0 (cero) se toman todas las muestras
TOKENS_ESPECIALES = {'[relleno]': 0, '[desconocido]': 1}
VALOR_DE_RELLENO = 0

## Carga de datos

Carga de datos de entrenamiento

In [3]:
df_entrenamiento = pd.read_csv(ARCHIVO_SET_DE_ENTRENAMIENTO)

print('Datos cargados con éxito, a continuación una muestra de los datos.')
df_entrenamiento.head()

Datos cargados con éxito, a continuación una muestra de los datos.


,title,category
0,Casita Muñecas Barbies Pintadas,DOLLHOUSES
1,Neceser Cromado Holográfico,TOILETRY_BAGS
2,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS
3,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS
4,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...,CAMERA_BATTERIES


Carga de datos de validación

In [5]:
df_validacion = pd.read_csv(ARCHIVO_SET_DE_VALIDACION)

print('Datos cargados con éxito, a continuación una muestra de los datos.')
df_validacion.head()

Datos cargados con éxito, a continuación una muestra de los datos.


,title,category
0,Metal Biela Dw10 Hdi 2.0,ENGINE_BEARINGS
1,Repuestos Martillo Rotoprcutor Bosch Gshsce Po...,ELECTRIC_DEMOLITION_HAMMERS
2,Pesca Caña Pejerrey Colony Brava 3m Fibra De V...,FISHING_RODS
3,Porcelanato Abitare Be 20x120 Cm. Ceramica Por...,PORCELAIN_TILES
4,Reconstruction Semi Di Lino Alfaparf Shampoo 1...,HAIR_SHAMPOOS_AND_CONDITIONERS


## Procesamiento de los títulos

Extracción de títulos del dataframe de entrenamiento.

In [6]:
if MUESTRAS > 0:
    titulos = df_entrenamiento.sample(MUESTRAS).title.to_list()
else:
    titulos = df_entrenamiento.title.to_list()

print('Títulos extraidos con éxito, a continuación una muestra de los mismos:\n')
for i in range(10):
    print(titulos[i])

Títulos extraidos con éxito, a continuación una muestra de los mismos:

Casita Muñecas Barbies Pintadas
Neceser Cromado Holográfico 
Funda Asiento A Medida D20 Chevrolet
Embrague Ford Focus One 1.8 8v Td (90cv) Desde 01-99
Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmcfx60n Fx60n
Gurgel Br 800
Harman Kardon Hk 3700 2ch Network Receiver _h
Pack Netbook´s
Olla Essen Duo
Teclado Mini Bluetooth


Extracción de los títulos del dataframe de validacion.

In [7]:
titulos_validacion = df_validacion.title.to_list()

print('Títulos extraidos con éxito, a continuación una muestra de los mismos:\n')
for i in range(10):
    print(titulos_validacion[i])

Títulos extraidos con éxito, a continuación una muestra de los mismos:

Metal Biela Dw10 Hdi 2.0
Repuestos Martillo Rotoprcutor Bosch Gshsce Por Separado
Pesca Caña Pejerrey Colony Brava 3m Fibra De Vidrio 7 Sec
Porcelanato Abitare Be 20x120 Cm. Ceramica Portinari
Reconstruction Semi Di Lino Alfaparf Shampoo 1000 Ml
Mascara Fotosensible Lüsqtoff, Oferta Y En Cuotas!
Bermuda John Cena 14/16 Original
20x Rueda Neumático Tuerca Set De 20 Lr068126 Oem Para Tierr
Pelota De Basquet Spalding Tf-elite Nº 6
Placard De Algarrobo Original 3 Puertas 


Armado de un corpus de títulos de entrenamiento. El corpus contiene los títulos procesados.

In [8]:
corpus_titulos = []
for titulo in tqdm(titulos, desc="Procesando títulos"):
    titulo_procesado = procesar_titulo(titulo)
    corpus_titulos.append(titulo_procesado)
# corpus_titulos contiene todos los títulos procesados.
# Cada título es una lista de palabras procesadas
print(f'Se han procesado {len(corpus_titulos)} títulos.\nMuestra de los 5 primeros:\n')
for i in range(5):
    print(corpus_titulos[i])

Procesando títulos:   0%|          | 0/4895280 [00:00<?, ?it/s]

Se han procesado 4895280 títulos.
Muestra de los 5 primeros:

['casita', 'muñecas', 'barbies', 'pintadas']
['neceser', 'cromado', 'holográfico']
['funda', 'asiento', 'medida', 'chevrolet']
['embrague', 'ford', 'focus']
['bateria', 'panasonic', 'dmwbcf', 'lumix', 'dmcfx']


Armado de un corpus de títulos de validación.

In [11]:
corpus_titulos_validacion = []
for titulo in tqdm(titulos_validacion, desc="Procesando títulos"):
    titulo_procesado = procesar_titulo(titulo)
    corpus_titulos_validacion.append(titulo_procesado)

print(f'Se han procesado {len(corpus_titulos_validacion)} títulos.\nMuestra de los 5 primeros:\n')
for i in range(5):
    print(corpus_titulos_validacion[i])

Procesando títulos:   0%|          | 0/1223820 [00:00<?, ?it/s]

Se han procesado 1223820 títulos.
Muestra de los 5 primeros:

['metal', 'biela']
['repuestos', 'martillo', 'rotoprcutor', 'bosch', 'gshsce', 'separado']
['pesca', 'caña', 'pejerrey', 'colony', 'brava', 'fibra', 'vidrio']
['porcelanato', 'abitare', 'ceramica', 'portinari']
['reconstruction', 'semi', 'lino', 'alfaparf', 'shampoo']


## Construcción del diccionario a partir del corpus de títulos

In [12]:
# https://radimrehurek.com/gensim/corpora/dictionary.html
print('Generando diccionario...')
diccionario = corpora.Dictionary(corpus_titulos)
diccionario.filter_extremes(no_below=2, no_above=1, keep_n=VOCAB_SIZE)
diccionario.patch_with_special_tokens(TOKENS_ESPECIALES)
diccionario.compactify()
print(f'Se generó un diccionario de longitud {len(diccionario)} a partir de {diccionario.num_docs} documentos.')

# Guardado del diccionario en archivo de texto
diccionario.save_as_text(ARCHIVO_DICCIONARIO, sort_by_word=True)
print(f'Se guardó el diccionario en {ARCHIVO_DICCIONARIO}.')

Generando diccionario...
Se generó un diccionario de longitud 50002 a partir de 4895280 documentos.
Se guardó el diccionario en ./data/diccionario.txt.


## Encoding de datos

Encoding de los títulos de entrenamiento.

In [13]:
encoded_titulos = []
for titulo in tqdm(corpus_titulos, desc='Encoding de títulos'):
    encoded_titulo = diccionario.doc2idx(titulo, unknown_word_index=1)
    encoded_titulos.append(encoded_titulo)
print('Se realizó el encoding de los títulos.\nMuestra de los 5 primeros:\n')
for i in range(5):
    print(encoded_titulos[i])

Encoding de títulos:   0%|          | 0/4895280 [00:00<?, ?it/s]

Se realizó el encoding de los títulos.
Muestra de los 5 primeros:

[50001, 2, 50000, 3]
[6, 4, 5]
[9, 7, 10, 8]
[11, 13, 12]
[14, 18, 16, 17, 15]


Encoding de los títulos de validación.

In [14]:
encoded_titulos_validacion = []
for titulo in tqdm(corpus_titulos_validacion, desc='Encoding de títulos'):
    encoded_titulo = diccionario.doc2idx(titulo, unknown_word_index=1)
    encoded_titulos_validacion.append(encoded_titulo)
    
print('Se realizó el encoding de los títulos.\nMuestra de los 5 primeros:\n')
for i in range(5):
    print(encoded_titulos_validacion[i])

Encoding de títulos:   0%|          | 0/1223820 [00:00<?, ?it/s]

Se realizó el encoding de los títulos.
Muestra de los 5 primeros:

[416, 1325]
[2799, 798, 1, 688, 1, 8400]
[629, 889, 1843, 3583, 3296, 1551, 1025]
[2438, 3964, 1260, 3965]
[1, 3017, 6739, 9836, 762]


## Completamiento de datos

Completamiento de los datos de entrenamiento.

In [15]:
print('Completamiento de datos.')
longitudes_titulos = [len(titulo) for titulo in encoded_titulos]
longitud_maxima = max(longitudes_titulos)
print(f'El título más largo tiene {longitud_maxima} palabras/indices.')
print(f'Se rellenó con {VALOR_DE_RELLENO} los valores faltantes en los títulos con menor longitud.\n')
data = [d[:ele] + [VALOR_DE_RELLENO] * (longitud_maxima - ele) for d, ele in zip(encoded_titulos, longitudes_titulos)]
for i in range(5):
    print(data[i])
X_train = torch.LongTensor(data)

Completamiento de datos.
El título más largo tiene 17 palabras/indices.
Se rellenó con 0 los valores faltantes en los títulos con menor longitud.

[50001, 2, 50000, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[6, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 7, 10, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[11, 13, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[14, 18, 16, 17, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Completamiento de los datos de validación.

In [17]:
print('Completamiento de datos.')
longitudes_titulos = [len(titulo) for titulo in encoded_titulos_validacion]
longitud_maxima = max(longitudes_titulos)
print(f'El título más largo tiene {longitud_maxima} palabras/indices.')
print(f'Se rellenó con {VALOR_DE_RELLENO} los valores faltantes en los títulos con menor longitud.\n')
data = [d[:ele] + [VALOR_DE_RELLENO] * (longitud_maxima - ele) for d, ele in zip(encoded_titulos_validacion, longitudes_titulos)]
for i in range(5):
    print(data[i])
X_val = torch.LongTensor(data)

Completamiento de datos.
El título más largo tiene 16 palabras/indices.
Se rellenó con 0 los valores faltantes en los títulos con menor longitud.

[416, 1325, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2799, 798, 1, 688, 1, 8400, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[629, 889, 1843, 3583, 3296, 1551, 1025, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2438, 3964, 1260, 3965, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 3017, 6739, 9836, 762, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Conversión de categorías a etiquetas

Función que convierte las categorías a etiquetas.

In [18]:
idx_to_target = sorted(df_entrenamiento["category"].unique())
target_to_idx = {t: i for i, t in enumerate(idx_to_target)}
def encode_target(target):
    # Convierte las categorías a etiquetas
    return target_to_idx[target]

Conversión de categorías a etiquetas del dataframe de entrenamiento.

In [19]:
categoria_etiquetada = [encode_target(t) for t in df_entrenamiento['category']]

print('Se etiquetaron las categorías.\nMuestra de las 5 primeras:')
for i in range(5):
    print(categoria_etiquetada[i])
y_train = torch.LongTensor(categoria_etiquetada)

Se etiquetaron las categorías.
Muestra de las 5 primeras:
188
570
116
25
103


Conversión de categorías a etiquetas del dataframe de validación.

In [21]:
categoria_validacion_etiquetada = [encode_target(t) for t in df_validacion['category']]

print('Se etiquetaron las categorías.\nMuestra de las 5 primeras:')
for i in range(5):
    print(categoria_validacion_etiquetada[i])
y_val = torch.LongTensor(categoria_validacion_etiquetada)

Se etiquetaron las categorías.
Muestra de las 5 primeras:
216
202
245
467
290


In [22]:
torch.save(X_train, './data/X_train.pt')
torch.save(y_train, './data/y_train.pt')

In [24]:
torch.save(X_val, './data/X_val.pt')
torch.save(y_val, './data/y_val.pt')

In [25]:
del df_entrenamiento

## Embedding de títulos

In [26]:
# https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding
vector_size = 300
embeddings_matrix = torch.randn(len(diccionario), vector_size)
embeddings_matrix[0] = torch.zeros(vector_size)
print(f'Tamaño de la matriz de embeddings: {embeddings_matrix.shape}.')
with bz2.open(ARCHIVO_DE_EMBEDDINGS, mode='rt') as file:
    for line in tqdm(file, total=1000654, desc="Recorriendo archivo de embeddings"):
        word, vector = line.strip().split(None, 1)
        if word in diccionario.token2id:
            embeddings_matrix[diccionario.token2id[word]] = torch.FloatTensor([float(n) for n in vector.split()])
embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                          padding_idx=0)
print(f'Finalizado el embedding de tamaño {embeddings.weight.shape}.')

Tamaño de la matriz de embeddings: torch.Size([50002, 300]).


Recorriendo archivo de embeddings:   0%|          | 0/1000654 [00:00<?, ?it/s]

Finalizado el embedding de tamaño torch.Size([50002, 300]).


In [27]:
torch.save(embeddings_matrix, './data/embeddings_matrix.pt')